In [91]:
import numpy as np
import pandas as pd

In [92]:
#read train csv and split
delimiter="\t"
df = pd.read_csv('train.csv',delimiter="\t").astype(str)

df.head(10)

,text,label
0,Get the latest from TODAY Sign up for our news...,1
1,2d Conan On The Funeral Trump Will Be Invited...,1
2,It’s safe to say that Instagram Stories has fa...,0
3,Much like a certain Amazon goddess with a lass...,0
4,At a time when the perfect outfit is just one ...,0
5,"When Reese Witherspoon was 17, she had already...",0
6,It’s official! WAGS Miami stars Ashley Nicole ...,0
7,"On March 24, 2018, one month and ten days afte...",1
8,A mosque in Georgia is under fire after they s...,1
9,Scott Disick is sticking up for Khloe Kardashi...,1


In [93]:
#find str in label
fliter = (df['label']== 'label')
df[fliter]

,text,label
1615,content,label


In [94]:
df= df.drop(index=1615)

In [95]:
data_x= df['text']
data_y= df['label']

In [96]:
#read test csv and split
delimiter="\t"
df2 = pd.read_csv('test.csv',delimiter="\t").astype(str)

df2.head(10)

,id,text
0,2,The 2017 Teen Choice Awards ceremony was held ...
1,3,"The concert, part of “The Joshua Tree Tour,” w..."
2,4,Selena Gomez refuses to talk to her mother abo...
3,5,This is worse than a lump of coal in your stoc...
4,6,Luann De Lesseps is going to rehab after her a...
5,7,Diamond's Weinstein meets a grisly end Saved ...
6,8,"After 13 years, Ashlee Simpson-Ross is once ag..."
7,9,"In honor of their 70th wedding anniversary, th..."
8,10,Demi Lovato has made no secret that she's a mi...
9,11,Twitter Caught Leaking Donald Trump's Private ...


In [97]:
test_x= df2['text']

In [98]:
df3=pd.read_csv('sample_submission.csv')
df3.head()

,id,label
0,2,1
1,3,1
2,4,0
3,5,0
4,6,0


In [99]:
test_y=df3['label']

In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [101]:
#tfidf weights
tfidf = TfidfVectorizer(stop_words='english')

In [102]:
#the text in train&test data turn into tfidf weight
data_x_tfidf = tfidf.fit_transform(data_x)
test_x_tfidf = tfidf.fit_transform(test_x)

In [103]:
data_x_tfidf 

<4986x63615 sparse matrix of type '<class 'numpy.float64'>'
	with 1029592 stored elements in Compressed Sparse Row format>

In [104]:
test_x_tfidf

<1247x32827 sparse matrix of type '<class 'numpy.float64'>'
	with 250815 stored elements in Compressed Sparse Row format>

In [105]:
x_train= data_x_tfidf #the weight of text in train data
y_train = data_y      #the label of text in train data

x_test= test_x_tfidf  #the weight of text in test data
y_test= test_y        #the label of text in test data

In [106]:
import xgboost as xgb
#build XGboost Classifier
xgb_statement = xgb.XGBClassifier()

In [107]:
#train XGboost Classifier
xgb_statement.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [108]:
# predict
y_xgb_statement_pred = xgb_statement.predict(x_test, validate_features=False)
print(y_xgb_statement_pred)

['0' '0' '0' ... '0' '0' '0']


In [109]:
# ground truth
print(y_test.values)

[1 1 0 ... 1 1 1]


In [110]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [111]:
y_true=y_test.values
y_pred=y_xgb_statement_pred
y_true.dtype

dtype('int64')

In [112]:
y_pred.dtype

dtype('O')

In [113]:
y_pred = y_pred.astype('int64')

In [115]:
print('accuracy：{:.2f}%'.format(accuracy_score(y_true, y_pred)*100))
print('precision：{}'.format(precision_score(y_true, y_pred, average=None)))
print('recall：{}'.format(recall_score(y_true, y_pred, average=None)))
print('F-measure：{}'.format(f1_score(y_true, y_pred, average=None)))

accuracy：50.12%
precision：[0.50342466 0.46835443]
recall：[0.93333333 0.05996759]
F-measure：[0.65406007 0.10632184]


In [116]:
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
#build LightGBM Classifier
lgb_statement = lgb.LGBMClassifier()

In [117]:
#turn into float32 for LightGBM
x_train2= data_x_tfidf.astype('float32')
y_train2 = data_y.astype('float32')

x_test2= test_x_tfidf.astype('float32')
y_test2= test_y.astype('float32')

In [118]:
#the format for feature of LightGBM
lgb_train = lgb.Dataset(x_train2, y_train2)
lgb_val = lgb.Dataset(x_test2, y_test2, reference=lgb_train)

In [119]:
#set params
params = {'max_depth': 5, 'min_data_in_leaf': 20, 'num_leaves': 35, 'objective':'binary'}
# train LightGBM
gbm = lgb.train(params, lgb_train)

In [120]:
# predict
y_lgb_statement_pred = gbm.predict(x_test2)
print(y_lgb_statement_pred)
np.where(y_lgb_statement_pred > 0.5,1,0)

[0.30996054 0.37839281 0.34647622 ... 0.30996054 0.30996054 0.34647622]


array([0, 0, 0, ..., 0, 0, 0])

In [121]:
y_true2=y_test2.values
y_pred2=np.where(y_lgb_statement_pred > 0.5,1,0)

In [122]:
y_true2.dtype

dtype('float32')

In [123]:
y_pred2.dtype

dtype('int32')

In [124]:
y_true2 = y_true2.astype('int32')

In [125]:
print('accuracy：{:.2f}%'.format(accuracy_score(y_true2, y_pred2)*100))
print('precision：{}'.format(precision_score(y_true2, y_pred2, average=None)))
print('recall：{}'.format(recall_score(y_true2, y_pred2, average=None)))
print('F-measure：{}'.format(f1_score(y_true2, y_pred2, average=None)))

accuracy：50.28%
precision：[0.50404531 0.36363636]
recall：[0.98888889 0.00648298]
F-measure：[0.66773848 0.01273885]
